In [53]:
import sys
sys.path.append('./src')
import syllo_gen
import random

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
all_possilbe = set()
for i in range(2):
    for j in range(6):
        real = random.choice([True, False])
        syllo, _ = syllo_gen.get_syllo(i+1, real)
        if syllo not in all_possilbe:
            all_possilbe.add(syllo)
            print(f'Story: {", ".join(syllo[:-1])} Question: {syllo[-1]}')
            print(f'Answer: {real}')
            print()

real = random.choice([True, False])
syllo, _ = syllo_gen.get_syllo(3, real)
print(f'Story: {", ".join(syllo[:-1])} Question: {syllo[-1]}')
print(f'Answer:')
print()
print(real)

Story: ∀x ([R2](x) → [R1](x)), ∀x ([R0](x) → ¬[R1](x)) Question: ∀x ([R0](x) → [R2](x))
Answer: False

Story: ∀x ([R1](x) → [R2](x)), ∃x ([R0](x) ∧ [R1](x)) Question: ∀x ([R0](x) → ¬[R2](x))
Answer: False

Story: ∃x ([R1](x) ∧ [R2](x)), ∀x ([R1](x) → [R0](x)) Question: ∃x ([R0](x) ∧ [R2](x))
Answer: True

Story: ∃x ([R1](x) ∧ ¬[R2](x)), ∀x ([R1](x) → [R0](x)) Question: ∀x ([R0](x) → [R2](x))
Answer: False

Story: ∀x ([R1](x) → [R2](x)), ∀x ([R0](x) → [R1](x)) Question: ∀x ([R0](x) → [R2](x))
Answer: True

Story: ∀x ([R2](x) → [R1](x)), ∀x ([R1](x) → [R0](x)) Question: ∃x ([R0](x) ∧ [R2](x))
Answer: True

Story: ∀x ([R1](x) → [R2](x)), ∃x ([R0](x) ∧ [R1](x)), ∀x ([R3](x) → ¬[R2](x)) Question: ∃x ([R0](x) ∧ ¬[R3](x))
Answer: True

Story: ∀x ([R1](x) → [R2](x)), ∀x ([R0](x) → [R1](x)), ∀x ([R0](x) → [R3](x)) Question: ∀x ([R3](x) → ¬[R2](x))
Answer: False

Story: ∀x ([R2](x) → ¬[R1](x)), ∃x ([R0](x) ∧ [R1](x)), ∀x ([R3](x) → [R2](x)) Question: ∃x ([R0](x) ∧ ¬[R3](x))
Answer: True

Story: 

In [84]:
real = random.choice([True, False])
syllo, _ = syllo_gen.get_syllo(1, real)
print(f'Story: {", ".join(syllo[:-1])} Question: {syllo[-1]}')
print(f'Output:')
print()
print(real)

Story: ∀x ([R1](x) → [R2](x)), ∀x ([R0](x) → [R1](x)) Question: ∀x ([R0](x) → [R2](x))
Output:

True


In [50]:
syllo, _ = syllo_gen.get_syllo(1, True)
print(syllo)
syllo[-1]

('∀x ([R1](x) → ¬[R2](x))', '∃x ([R1](x) ∧ [R0](x))', '∃x ([R0](x) ∧ ¬[R2](x))')


'∃x ([R0](x) ∧ ¬[R2](x))'

In [304]:
all_possilbe = set()
for i in range(3):
    for j in range(200**(i+1)):
        syllo, _ = syllo_gen.get_syllo(i+1, True)
        all_possilbe.add(tuple(syllo))

for i in [4,5]:
    for j in range(200**3):
        syllo, _ = syllo_gen.get_syllo(i, True)
        all_possilbe.add(tuple(syllo))


('∀x ([R2](x) → [R1](x))',
 '∀x ([R1](x) → [R0](x))',
 '∃x ([R0](x) ∧ [R2](x))',
 '∀x ([R0](x) → [R3](x))',
 '∃x ([R3](x) ∧ [R2](x))')

In [299]:
all_possilbe = list(all_possilbe)
all_possilbe = [list(syllo) for syllo in all_possilbe]
all_possilbe = sorted(all_possilbe, key=lambda x: len(x))

['∀x ([R2](x) → ¬[R1](x))',
 '∀x ([R0](x) → [R1](x))',
 '∃x ([R0](x) ∧ ¬[R2](x))',
 '∀x ([R3](x) → [R2](x))',
 '∃x ([R0](x) ∧ ¬[R3](x))']

In [300]:
syllo_gen.resolve_predicate(all_vars)

{'[R3]': '⊕', '[R0]': '∨', '[R1]': '⊕', '[R2]': '∨'}

In [ ]:
# TODO: Add grounded statements